# Image Data Augmentation Using Deep Convolutional Generative Adversarial Network (DCGAN)
# Models

## Load Libraries

In [0]:
%matplotlib inline

import numpy as np
import keras
from keras import layers
import myUtilities as myUtils
import sklearn.utils

Using TensorFlow backend.


Using TensorFlow backend.


## Generator Model

Follows the DCGAN model from Deep Learning with Python by Francis Chollet

In [2]:
latent_dim = 128
height = 128
width = 128
channels = 5

generator_input = keras.Input(shape=(latent_dim,))

# First, transform the input into a 16x16 128-channels feature map
x = layers.Dense(256 * 64 * 64)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((64, 64, 256))(x)

# Then, add a convolution layer
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Upsample to 32x32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

# Few more conv layers
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Produce a 32x32 1-channel feature map
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1048576)           135266304 
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1048576)           0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 64, 64, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 256)       1638656   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 128, 128, 256)     1048832   
__________

## Discriminator Model

In [3]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

# Here, I have added additional convolutions layers with stride 2 to further reduce
# the size of the output before the final dense layer.
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)

# One dropout layer - important trick!
x = layers.Dropout(0.4)(x)

# Classification layer
x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

# To stabilize training, we use learning rate decay
# and gradient clipping (by value) in the optimizer.
# discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
# Reduce the learning rate so that the discriminator and generator does not overpower
# each other quickly. At larger learning rates, the loss of one drops to zero while the
# other shows no change. 
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 5)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 126, 126, 128)     5888      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 126, 126, 128)     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 62, 62, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 30, 30, 128)       262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 30, 30, 128)       0         
__________

## The Adversarial Network Model

In [4]:
# Set discriminator weights to non-trainable
# (will only apply to the `gan` model)
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

# Reduce the learning rate to prevent the networks loss from settling
# into a local minimum. See discriminator learning rate.
# gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan_optimizer = keras.optimizers.RMSprop(lr=0.0002, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

## Training the Model

In [5]:
# Load data
x_train = myUtils.load_preprocessed_data()

# Normalize data
x_train = x_train.astype('float32') / 255.

iterations = 10000
batch_size = 20
save_dir = '.\data\\gan_images\\'

# Start training loop
start = 0
for step in range(iterations):
    # Sample random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # Decode them to fake images
    generated_images = generator.predict(random_latent_vectors)

    # Combine them with real images
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # Assemble labels discriminating real from fake images
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])

    # Add random noise to the labels - important trick!
    labels += 0.05 * np.random.random(labels.shape)

    # Train the discriminator
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # sample random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # Assemble labels that say "all real images"
    misleading_targets = np.zeros((batch_size, 1))

    # Train the generator (via the gan model,
    # where the discriminator weights are frozen)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
        # Is shuffling needed for stochastic gradient descent?
        #x_train = sklearn.utils.shuffle(x_train)

    # Occasionally save / plot
    if step % 50 == 0:

        # Print metrics
        print('discriminator loss at step %s: %s' % (step, d_loss))
        print('adversarial loss at step %s: %s' % (step, a_loss))

        # Save one generated image
        # img = image.array_to_img(generated_images[0] * 255., scale=False)
        # img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))
        np.save(save_dir + 'generated_' + str(step), generated_images[0] * 255.)

        # Save one real image, for comparison
        # img = image.array_to_img(real_images[0] * 255., scale=False)
        # img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))
        np.save(save_dir + 'real_' + str(step), real_images[0] * 255.)
        
    if step % 1000 == 0:

        # Save model weights
        gan.save_weights('.\data\\gan_model\\gan' + str(step) + '.h5')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\keras\engine\training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


discriminator loss at step 0: 0.6886867
adversarial loss at step 0: 0.6797119


discriminator loss at step 50: 0.66255397
adversarial loss at step 50: 0.66550463


discriminator loss at step 100: 0.5520548
adversarial loss at step 100: 0.7829113


discriminator loss at step 150: 0.7019726
adversarial loss at step 150: 0.71089953


...

discriminator loss at step 5300: 0.020768289
adversarial loss at step 5300: 9.133642


discriminator loss at step 5350: 0.16729814
adversarial loss at step 5350: 3.096107


discriminator loss at step 5400: 0.047843087
adversarial loss at step 5400: 4.035048


discriminator loss at step 5450: 0.015765142
adversarial loss at step 5450: 6.402471


InternalError: GPU sync failed